<a href="https://colab.research.google.com/github/KripaMishra/LangChain/blob/main/Q%26A_wo_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

In [16]:
from google.colab import userdata

In [21]:
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 15.0 MB/s eta 0:00:00


In [17]:
OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")

In [18]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
from langchain_community.embeddings import GPT4AllEmbeddings

In [22]:
gpt4all_embd = GPT4AllEmbeddings()

Downloading: 100%|██████████| 45.9M/45.9M [00:01<00:00, 38.6MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 178MiB/s]


In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_API_KEY)

In [25]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [26]:
docs= loader.load()

In [27]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [28]:
splits=text_splitter.split_documents(docs)

In [29]:
vectorstore=Chroma.from_documents(documents=splits, embedding=gpt4all_embd)

In [30]:
retriever=vectorstore.as_retriever() # you can also give the "search type" argument and "search kwargs" to define how many results to retrieve
prompt=hub.pull("rlm/rag-prompt")

In [31]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps. It helps agents plan ahead and tackle tasks more effectively. This process can be done through prompting techniques, task-specific instructions, or human inputs.'

In [34]:
rag_chain.invoke("what are agent systems")

"Agent systems are autonomous systems powered by LLM, with LLM serving as the agent's brain. These systems use planning and memory components to break down tasks into subgoals, reflect on past actions, and improve future results. Current agent systems rely on natural language interfaces, but may face reliability issues with model outputs."

In [35]:
rag_chain.invoke("explain the Tree of Thoughts")

'The Tree of Thoughts extends the Chain of Thought (CoT) technique by exploring multiple reasoning possibilities at each step, generating multiple thoughts per step to create a tree structure. Task decomposition can be done using LLM with simple prompting, task-specific instructions, or human inputs. The search process can be BFS or DFS with each state evaluated by a classifier or majority vote.'

In [36]:
rag_chain.invoke("who won the wrestlemania 40 main event")

"I don't know."

In [37]:
# cleanup
vectorstore.delete_collection()